# VacationPy
----

## Analysis
The difference between the hemispheres is interesting since these regression models have the least variance since the data points are closer to the regression line.  When comparing the plot of Max Temp vs. Latitude for each region, there is not as much symmetry as assumed initially. The hypothesis was that the hemispheres would like a rainbow, and as locations progressed away from a latitude of 0, those temperatures would mirror each other.  However, there is still a substantial amount of similarity that is worth exploring. 

### #1: Symmetry
Firstly, the assumed rainbow shape appears in the figure comparing city Latitude to Max Temperature.  The peak of the rainbow isn't at a latitude of 0 but a 20. This peak suggests that the equator is not the middle of the parallels between the northern and the southern hemispheres. However, there are more data points greater than 0, so the symmetry could be interrupted by the sample size.

### #2: Correlations
Second, the r-squared value for Southern Hemisphere is .67 compared to the r-squared value of .49 for the Northern Hemisphere.  The Southern's coefficient is .82, which means there is a relatively strong positive relationship and is slightly higher than Northern's coefficient of -.7, which is a moderately negative relationship. The Northern has outliers of warmer temperatures between latitude 20 and 50, with outliers of cooler temperatures around the latitude of 5 and again around 70.   This comparison suggests that the data points better to fit the line for the Southern Hemisphere.  This difference is most likely a continuation fo the previous observation.

### #3: Why Temperature
Lastly, the other linear regression plots do not suggest any positive or negative relationships. The lack of correlation is most likely because the sun does not directly impact those other factors like humidity and wind. Those other factors might most likely be influenced by temperature (of which this analysis does not support) if heat affects moisture and pressure.  Since the sun's position influences seasonal temperatures of locations at different latitudes, it would seem appropriate that there would be a stronger correlation.  It would be interesting to plot data based on latitude over 20 compared to under 20 to see if those values fit the line better and increase the relationship's strength or perhaps increase the sample size to see what impact that has on the data.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported
path="output_data\city_data.csv"

df=pd.read_csv(path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)

rating = df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center=(25, 10), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=99,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# create new data frame for narrowed down cities
criteria = df.loc[(df['Humidity'] <= 76) & (df['Max Temp'] < 84) & (df['Max Temp'] >= 75) & (df['Cloudiness'] < 20)].head(10)
criteria.dropna()

,Unnamed: 0,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
4,5,Airai,-8.93,125.41,76.55,29.0,3.0,3.49,TL
17,21,Assiut,27.18,31.18,80.60,47.0,0.0,5.82,EG
29,34,Karratha,-20.74,116.85,76.23,23.0,0.0,16.44,AU
30,35,Luwuk,-0.95,122.79,83.43,60.0,5.0,5.79,ID
140,158,Alta Floresta,-9.88,-56.09,77.00,27.0,0.0,0.34,BR
152,170,Gizo Government Station,-8.10,156.84,83.84,69.0,18.0,0.81,SB
163,183,Yeppoon,-23.13,150.73,77.00,16.0,6.0,12.75,AU
183,208,Aswān,24.09,32.91,82.81,24.0,0.0,7.20,EG
224,250,Jacareacanga,-6.22,-57.75,80.35,42.0,0.0,2.19,BR
225,251,Atambua,-9.11,124.89,83.05,42.0,0.0,0.58,ID


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df.
hotel_df = criteria[["City Name","Lat","Lng","Country"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters.
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# Hit the Google Places API for each city's coordinates.
# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lon = row['Lng']

    # set up a parameters dictionary
    params = {
    "location": (f'{lat},{lon}'),
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

    # make request
    response = requests.get(base_url, params=params).json()
    
    # Store the first Hotel result into the DataFrame.
    try:
    
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None"
        print(f"Missing field/result... skipping {index}.")

C:\Users\tawnyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\tawnyn\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping 4.


In [6]:
# Store the first Hotel result into the DataFrame
hotel_df["City"] = hotel_df["City Name"]
hotel_df

C:\Users\tawnyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City Name,Lat,Lng,Country,Hotel Name,City
4,Airai,-8.93,125.41,TL,None,Airai
17,Assiut,27.18,31.18,EG,Florence Hotel Assiut,Assiut
29,Karratha,-20.74,116.85,AU,Best Western Plus The Ranges Karratha,Karratha
30,Luwuk,-0.95,122.79,ID,Swiss-Belinn Hotel Luwuk,Luwuk
140,Alta Floresta,-9.88,-56.09,BR,Lisboa Palace Hotel,Alta Floresta
152,Gizo Government Station,-8.10,156.84,SB,Gizo Hotel,Gizo Government Station
163,Yeppoon,-23.13,150.73,AU,Yeppoon Beach House,Yeppoon
183,Aswān,24.09,32.91,EG,Sofitel Legend Old Cataract Aswan,Aswān
224,Jacareacanga,-6.22,-57.75,BR,HOTEL VIANA,Jacareacanga
225,Atambua,-9.11,124.89,ID,Hotel King Star,Atambua


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

# Add layers
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))